# Calculating the Hessian on simulator

In [1]:
import pennylane as qml
import numpy as np
from qiskit import IBMQ
import itertools
import matplotlib.pyplot as plt

In [2]:
datafile = "results_sim.pickle"

## Hardware-friendly circuit

In [3]:
n_wires = 5

In [4]:
dev = qml.device("default.qubit", wires=n_wires)

In [5]:
@qml.qnode(dev)
def layers(weights):
    for i in range(n_wires):
        qml.RX(weights[i], wires=i)

    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[2, 1])
    qml.CNOT(wires=[3, 1])
    qml.CNOT(wires=[4, 3])
    return qml.expval(qml.PauliZ(1))

In [6]:
seed = 2

weights = qml.init.basic_entangler_layers_uniform(n_layers=1, n_wires=5, seed=seed).flatten()
weights

tensor([2.73943676, 0.16289932, 3.4536312 , 2.73521126, 2.6412488 ], requires_grad=True)

In [7]:
grad = qml.grad(layers, argnum=0)

In [8]:
layers(weights)

-0.7938055593697134

In [9]:
np.round(grad(weights), 7)

array([-0.3376347,  0.1304665,  0.2560632, -0.3416029,  0.       ])

## Calculating the Hessian

In [10]:
import pickle

In [11]:
s = 0.5 * np.pi
denom = 4 * np.sin(s) ** 2
shift = np.eye(len(weights))


def hess_gen_results(weights):
    
    try:
        with open(datafile, "rb") as f:
            results = pickle.load(f)
    except:
        results = {}
    
    for c in itertools.combinations(range(len(weights)), r=2):
        print(c)
        if not results.get(c):
            weights_pp = weights + s * (shift[c[0]] + shift[c[1]])
            weights_pm = weights + s * (shift[c[0]] - shift[c[1]])
            weights_mp = weights - s * (shift[c[0]] - shift[c[1]])
            weights_mm = weights - s * (shift[c[0]] + shift[c[1]])

            f_pp = layers(weights_pp)
            f_pm = layers(weights_pm)
            f_mp = layers(weights_mp)
            f_mm = layers(weights_mm)
            results[c] = (f_pp, f_mp, f_pm, f_mm)

            with open(datafile, "wb") as f:
                pickle.dump(results, f)
    
    for i in range(len(weights)):
        print((i, i))
        if not results.get((i, i)):
            f_p = layers(weights + 0.5 * np.pi * shift[i])
            f_m = layers(weights - 0.5 * np.pi * shift[i])
            f = layers(weights)

            results[(i, i)] = (f_p, f_m, f)

            with open(datafile, "wb") as f:
                pickle.dump(results, f)
        

def get_hess(weights):
    hess = np.zeros((len(weights), len(weights)))
    
    with open(datafile, "rb") as f:
        results = pickle.load(f)
    
    for c in itertools.combinations(range(len(weights)), r=2):
        r = results[c]
        hess[c] = (r[0] - r[1] - r[2] + r[3]) / denom
    
    hess = hess + hess.T
    
    for i in range(len(weights)):
        r = results[(i, i)]
        hess[i, i] = (r[0] + r[1] - 2 * r[2]) / 2
    
    return hess       

In [12]:
hess_gen_results(weights)

(0, 1)
(0, 2)
(0, 3)
(0, 4)
(1, 2)
(1, 3)
(1, 4)
(2, 3)
(2, 4)
(3, 4)
(0, 0)
(1, 1)
(2, 2)
(3, 3)
(4, 4)


In [13]:
hess = get_hess(weights)
np.around(hess, 3)

array([[ 0.794,  0.055,  0.109, -0.145,  0.   ],
       [ 0.055,  0.794, -0.042,  0.056, -0.   ],
       [ 0.109, -0.042,  0.794,  0.11 ,  0.   ],
       [-0.145,  0.056,  0.11 ,  0.794,  0.   ],
       [ 0.   , -0.   ,  0.   ,  0.   , -0.   ]])